In [68]:
# -*- coding: utf-8 -*-
"""
Created on Mon May  1 11:51:33 2023

author: Sara Miller
calculate district level stats for RHEAS vs preharvest survey data for Zambia
select best cultivars for each district
"""

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import sys
import glob
import re

In [69]:
# set working directory
wdir = r'D:\RONO\rheas\maize_crop_modeling\crop-modeling\zambia\data\cultivartests'
# wdir ='data/cultivartests/cultivars/'


In [71]:
##############################################################################
# first, loop through all CSV files containing the different cultivars and years
# then append all data to one CSV

# get list of csvs containing cultivar test results
# these files should be in a sub-folder in your working directory called 'csvs'
flist = glob.glob(wdir+'cultivar*.csv')
df = pd.DataFrame()
for f in flist:
    df1 = pd.read_csv(f)
    # get cultivar number and year from file name, add to csv
    df1['cultivar'] = re.search(r'cultivar(.*)_',f).group(1)
    year = re.search('_(.*).csv',f).group(1)
    df1['year'] = int(year)
#     # make sure that only yields from the selected year are added
#     df1['planting'] = pd.to_datetime(df1['planting'])
#     df1 = df1.set_index('planting')
#     df1['PlantingYear'] = df1.index.year.values
#     df1['PlantingMonth'] = df1.index.month.values
#     df1 = df1.reset_index()
#     df1 = df1.loc[((df1['PlantingYear']==int(year))&(df1['PlantingMonth']>8))]
#     # append to one single dataframe and save to a CSV
#     df = pd.concat([df,df1],ignore_index=True)
    
# df.to_csv(wdir+'allcultivars.csv')
df1


,planting,cname,harvest,gwad,cultivar,year,PlantingYear,PlantingMonth
0,2021-10-15,Chadiza,2022-01-20,4227,tests/cultivars\cultivar10,2021,2021,10
1,2021-10-15,Chadiza,2022-01-20,4221,tests/cultivars\cultivar10,2021,2021,10
2,2021-10-15,Chadiza,2022-01-20,4338,tests/cultivars\cultivar10,2021,2021,10
3,2021-10-16,Chadiza,2022-01-22,4363,tests/cultivars\cultivar10,2021,2021,10
4,2021-10-16,Chadiza,2022-01-22,4219,tests/cultivars\cultivar10,2021,2021,10
...,...,...,...,...,...,...,...,...
11544,2021-11-11,Zimba,2022-03-01,5338,tests/cultivars\cultivar10,2021,2021,11
11545,2021-11-11,Zimba,2022-03-01,5372,tests/cultivars\cultivar10,2021,2021,11
11546,2021-11-11,Zimba,2022-03-01,5110,tests/cultivars\cultivar10,2021,2021,11
11547,2021-11-12,Zimba,2022-03-02,5123,tests/cultivars\cultivar10,2021,2021,11


In [67]:
for f in flist:
    print(f)

In [30]:
###############################################################################

# read in csvs with dssat outputs, survey data
pre = pd.read_csv(wdir+'zambia_preharvest_data_clean.csv')
df = pd.read_csv(wdir+'allcultivars.csv')

# optionally: remove anomalous yield values from the preharvest dataset before comparing to DSSAT
#pre = pre.loc[pre['flag']=!2]

df = df.rename(columns={'cname':'district'})

df = df.reset_index()
stats = pd.DataFrame()

for c in df.cultivar.unique():
    for p in df.district.unique():
        pre1 = pre.loc[pre['district']==p]
        df2 = df.loc[((df['district']==p)&(df['cultivar']==c))]

        # get average season length in days for each cultivar
        try:
            df2['harvest'] = pd.to_datetime(df2['harvest'])
            df2['planting'] = pd.to_datetime(df2['planting'])
            
            df2['slength'] = (df2['harvest']-df2['planting']).dt.days
            slength = df2['slength'].mean()
        except:
            slength = np.NaN

        
        pre1 = pre1.set_index(['district','year'])
        df2 = df2.groupby(['district','year']).mean()
        
        # combine the two dataframes: note that here the yield is stored in column 'gwad' for dssat
        # and column 'yield' for the survey data
        # note that if there are years that one dataset has data for that the other doesn't
        # the years that are not present will be filled with NaNs
        df1 = pd.concat([pre1,df2],axis=1)
        
        # calculate statistics one province at a time and append to a new dataframe
        # correlation
        corr = df1['gwad'].corr(df1['yield'])
        # root mean square error
        rmse = np.sqrt(((df1['gwad']-df1['yield'])**2).mean())
        # percentage rmse
        perrmse = np.sqrt(((df1['gwad']-df1['yield'])**2).mean())/(df1['yield'].mean())*100
        # unbiased rmse
        ubrmse = np.sqrt((((df1['gwad']-df1['gwad'].mean())-(df1['yield']-df1['yield'].mean()))**2).mean())
        # bias
        bias = (df1['gwad']-df1['yield']).mean()
        stats = stats.append({'district':p,'cultivar':c, 'rmse':rmse, '%rmse':perrmse,'unbiased rmse':ubrmse, 'bias':bias, 'correlation':corr, 'season length':slength}, ignore_index=True)

# save statistics as a csv
stats.to_csv(wdir+'districteval.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/cultivartests/cultivarszambia_preharvest_data_clean.csv'